In [5]:
import os
import numpy as np
import pandas as pd
import numpy as np
from math import log
from sklearn.model_selection import train_test_split as tts
from scipy.stats import multivariate_normal as mnorm


In [6]:
PATHS = [
    '/home/shubham/MAIN/Academics/CS-669/assignment_01/linearly_seperable_data',
    '/home/shubham/MAIN/Academics/CS-669/assignment_01/non_linearly_seperable data',
    '/home/shubham/MAIN/Academics/CS-669/assignment_01/real_world_data'
]

In [7]:
def getGaussian(x,mean,sig):
    return -(((x-mean)**2)/sig**2 + 2*log(sig))

class bayesClassifier:

    def __init__(self):
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
        self.X = None
        self.Y = None
        self.infoMatrix = None
        self.fitted = False
        self.keys = None
        self.ce = False
        self.prior = None
    
    
    def fit(self,X, Y):
        self.X, self.Y = X, Y
        self.X_train, self.X_test, self.y_train, self.y_test = np.array(tts(X,Y,test_size=0.3, random_state=42))
        self.y_train = np.array(self.y_train)
        self.X_train = np.array(self.X_train)
        self.X_test = np.array(self.X_test)
        self.y_test = np.array(self.y_test)
        self.y_dim, self.prior = np.unique(self.y_train, return_counts=True)
        self.prior = self.prior / sum(self.prior)
        self.infoMatrix = np.empty((len(set(self.Y)),len(self.X[0]),2),dtype=float)         #[[[m11,sig11],[m12,sig12]],[[m21,sig21],[m22,sig22]]]
        
        self.fitted = True
        
        self.calInfo()
    
    
    def calInfo(self):
        if self.fitted:
            mp = {}
            
            for i in range(len(self.X_train)):
                if self.y_train[i] in mp:
                    mp[self.y_train[i]].append(self.X_train[i])
                else:
                    mp[self.y_train[i]] = [self.X_train[i]]
            
            self.keys = list(mp.keys())

            self.info = {}

            for k in self.keys:
                self.info[k] = [np.mean(mp[k],axis=0),np.cov(np.array(mp[k]).T)]
            '''
            for j in range(len(self.keys)):
                l = np.mean(mp[self.keys[j]],axis=0)
                l2 = np.std(mp[self.keys[j]],axis=0)
                for i in range(len(self.X_train[0])):
                    self.infoMatrix[j][i][0] = l[i]
                    self.infoMatrix[j][i][1] = l2[i]
            '''
        else:
            raise Exception("Please fit the data into the model first.")
    
    
    
    def predict(self,x):
        post = []
        for i in range(len(self.keys)):            
            l = mnorm.pdf(x,mean=self.info[self.keys[i]][0],cov=self.info[self.keys[i]][1], allow_singular=True)
            post.append(l*self.prior[self.keys[i]])
        label = np.argmax(post)
        confidence = post[label]
        return label, confidence
    
    def geteff(self):
        self.confMat = [ len(self.y_dim)*[0] for i in range(len(self.y_dim))]
        for i in range(len(self.X_test)):
            y_pred = self.predict(self.X_test[i])[0]
            self.confMat[y_pred][self.y_test[i]] += 1
                
        acc = 0
        for i in range(len(self.confMat)):
            acc += self.confMat[i][i]
        
        return acc/len(self.X_test) 
    


In [8]:
def solve_for_data(data):
    model = bayesClassifier()
    model.fit(data[:, :-1], data[:,-1].astype(int))
    print(f'prior : {model.prior}')
    print(f'Efficiency : {model.geteff()}')
    

### For linearly seperable data

In [9]:
def get_data(file_path, label):
    with open(file_path) as f:
        features = np.array(list(map(lambda x : list(map(float, x.strip().split())),f.readlines())))
        labels = np.reshape(np.array(features.shape[0]*[label]), [-1, 1])
        features = np.append(features, labels, axis=1)
    return features

data1 = np.append(get_data(PATHS[0] + '/Class1.txt', 0), get_data(PATHS[0] + '/Class2.txt', 1),0)
data2 = np.append(get_data(PATHS[1] + '/Class1.txt', 0), get_data(PATHS[1] + '/Class2.txt', 1),0)
data3 = np.append(get_data(PATHS[2] + '/class1.txt', 0), get_data(PATHS[2] + '/class2.txt', 1),0)
data3 = np.append(data3, get_data(PATHS[2] + '/class3.txt', 2), 0)

In [11]:
data1.shape

(2000, 3)

In [303]:
solve_for_data(data1)

prior : [0.49857143 0.50142857]
Efficiency : 1.0


In [304]:
solve_for_data(data2)

prior : [0.49857143 0.50142857]
Efficiency : 0.995


In [306]:
solve_for_data(data3)

prior : [0.34446764 0.31816284 0.33736952]
Efficiency : 0.9079396005845105


###  For non-linearly separable